In [89]:
import pennylane as qml
import tensorflow as tf
import numpy as np

In [4]:
tf.__version__

'2.1.0'

In [75]:
dev = qml.device("default.qubit", wires = 2, analytic = False, shots = 100)
N=2
def circuit(theta):
    qml.RX(theta[0], wires=0)
    qml.RX(theta[1], wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0))

In [76]:
theta = tf.Variable([0.5, 0.5], dtype=tf.float64)

In [78]:
q_circuit = qml.QNode(circuit, dev, diff_method="parameter-shift").to_tf()
q_circuit(theta)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9>

In [130]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.1)

for i in range(100):
    with tf.GradientTape() as tape:
        loss = q_circuit(theta)

    grad = tape.gradient(loss, [theta])
    opt.apply_gradients(zip(grad, [theta]))

In [80]:
q_circuit(theta)

<tf.Tensor: shape=(), dtype=float64, numpy=-1.0>

## circuit with single shots (does not work)

In [246]:
wires = 4
layers = 4
dev2 = qml.device("default.qubit", wires=wires)

In [247]:
def Barren_circuit(params, **kwargs):
    for j in range(layers):
        for i in range(wires):
            qml.Rot(*params[j][i], wires=i)
        for i in range(wires):
            qml.CNOT(wires=[i, (i+1)%wires])

In [248]:
obs = [qml.PauliZ(i)@qml.PauliZ((i+1)%wires) for i in range(wires)]
coeffs = np.random.rand(len(obs))
H = qml.vqe.Hamiltonian(coeffs, obs)

In [249]:
cost = qml.VQECost(Barren_circuit, H, dev2, interface="tf")

In [250]:
params = tf.Variable(np.random.rand(layers, wires, 3), dtype=tf.float64)

cost(params)

<tf.Tensor: shape=(), dtype=float64, numpy=0.3870578696083652>

In [251]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.1)

for i in range(100):
    with tf.GradientTape() as tape:
        loss = cost(params)

    print(loss.numpy())
    grad = tape.gradient(loss, [params])
    opt.apply_gradients(zip(grad, [params]))

0.3870578696083652
0.3475259725065135
0.3071128835591124
0.26608600811127414
0.22471966801438548
0.18329258346482505
0.14208603234878456
0.10138128186692835
0.06145497002818362
0.022571713995358005
-0.015025842748037574
-0.0511283750243612
-0.08557079187507377


KeyboardInterrupt: 

In [190]:
cost(params)

<tf.Tensor: shape=(), dtype=float64, numpy=-2.2714678319254347>

## Gradients with samples

In [407]:
dev = qml.device("default.qubit", wires = 2, analytic = False, shots = 100)
N=2
def circuit(theta):
    qml.Hadamard(wires=0)
    qml.RX(theta[0], wires=0)
    qml.RX(theta[1], wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0))

In [408]:
theta = tf.Variable([0.5, 0.5], dtype=tf.float64)

In [409]:
q_circuit = qml.QNode(circuit, dev).to_tf()
q_circuit(theta)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.14>

In [410]:
tf.reduce_mean(loss)

<tf.Tensor: shape=(), dtype=float64, numpy=0.2>

In [411]:
l = 0
for i in range(100):
    l += q_circuit(theta)
    
l

<tf.Tensor: shape=(), dtype=float64, numpy=-1.3399999999999994>

In [412]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.1)

for i in range(100):
    loss = 0
    with tf.GradientTape() as tape:
        loss =   q_circuit(theta)
    
    print(loss.numpy())
    grad = tape.gradient(loss, [theta])
    print(grad)
    opt.apply_gradients(zip(grad, [theta]))
    print(theta)

-0.04
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.  , 0.06])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.5  , 0.494])>
-0.12
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.09, -0.02])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.509, 0.496])>
-0.04
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.01, -0.02])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.51 , 0.498])>
0.14
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.03,  0.04])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.513, 0.494])>
-0.12
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.01,  0.01])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.514, 0.493])>
-0.24
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.09, -0.06])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.523, 0.499])>
0.0
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.0

[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.04,  0.14])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.532, 0.561])>
-0.02
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.09, 0.03])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.523, 0.558])>
0.1
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.  , 0.13])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.523, 0.545])>
0.08
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.07, 0.02])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.516, 0.543])>
-0.02
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([-0.03, -0.02])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.519, 0.545])>
-0.18
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([ 0.05, -0.06])>]
<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.514, 0.551])>
0.0
[<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.07, 0.07])>]
<